In [2]:
pip install diff-match-patch

     |████████████████████████████████| 71kB 4.2MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/WikiRevisions

/content/drive/My Drive/WikiRevisions


In [5]:
import xml.etree.ElementTree as ET
import time
from diff_match_patch import diff_match_patch
import os

In [6]:
tree = ET.parse('The_Weeknd.knolml')
root = tree.getroot()
revisions=[]
for instance in root.iter('Instance'):
  for text in instance.iter('Text'):
    revisions.append(text.text)


In [7]:
def diff_match(text1,text2):
  dmp = diff_match_patch()
  start = time.time()
  diff = dmp.patch_make(text1,text2)
  diff = dmp.patch_toText(diff)
  end = time.time()
  return (end-start) , diff

In [8]:
def patch_match(text1,diff):
  dmp = diff_match_patch()
  start = time.time()
  patches = dmp.patch_fromText(diff)
  new_text, _ = dmp.patch_apply(patches, text1)
  end = time.time()
  return (end-start),new_text

In [10]:
temp1 = time.time()
temp2 = time.time()
temptime =  temp2-temp1
diff_time_k=[]
patch_time_k = []
file_size_k = []
for k in range(1,10,1):
  diff_list = []
  diff_time = []
  patch_time = []
  patch_list = []
  patch_files = []
  for i in range(len(revisions)):
    if (i%k==0):
      diff_list.append(revisions[i])
      diff_time.append(temptime)
    else :
      prev_text = revisions[i-1]
      cur_text= revisions[i]
      t , diff = diff_match(prev_text,cur_text)
      diff_list.append(diff)
      diff_time.append(t)
  for i in range(len(revisions)):
    if i%k==0:
      temp = int(i/k)
      current_text = diff_list[k*temp]
      patch_time.append(temptime)
      patch_files.append(current_text)
    else:
      temp = int(i/k)
      patched_text = diff_list[k*temp]
      total = 0
      for j in range(temp*k,i,1):
        t,patched_text = patch_match(patched_text,diff_list[j+1])
        total+=t
      patch_time.append(total)
      patch_files.append(patched_text)

  total_time_diff=0
  total_time_patch=0
  l = ""
  for i in range(len(diff_time)):
    total_time_diff +=diff_time[i]
    total_time_patch +=patch_time[i]
    l+=diff_list[i]
  diff_time_k.append(total_time_diff)
  patch_time_k.append(total_time_patch)
  file_size_k.append(len(l.encode('utf-8')))




In [23]:
print ("k","diff_time      ","patch_time   ","total_time","all_file_size")
for i in range(1,10,1):
  print(i,diff_time_k[i-1],patch_time_k[i-1],diff_time_k[i-1]+patch_time_k[i-1],file_size_k[i-1])


k diff_time       patch_time    total_time all_file_size
1 0.2045118808746338 0.2045118808746338 0.4090237617492676 310828689
2 39.67944002151489 0.6199462413787842 40.29938626289368 157256182
3 51.336711168289185 0.9138867855072021 52.25059795379639 106097651
4 64.94996523857117 1.2987871170043945 66.24875235557556 80631236
5 64.90843963623047 1.6985747814178467 66.60701441764832 64967863
6 62.491668701171875 1.949343204498291 64.44101190567017 54795765
7 64.50891828536987 2.3565833568573 66.86550164222717 47490070
8 70.86159420013428 2.6599719524383545 73.52156615257263 42250222
9 71.08018898963928 2.9799697399139404 74.06015872955322 37671383
